## Main Code

### 0. Libraries

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
import json
import time
import pandas as pd
from datetime import timedelta

/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.cloud.resourcemanager_v3 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.cloud.resourcemanager_v3 past that date.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/google/api_core/_python_version_support.py:275: FutureWarn

### 1. Configuration & Naming Standards

In [ ]:
# ==========================================
# 1. CONFIGURATION & NAMING STANDARDS
# ==========================================
PROJECT_ID = "project-nirvana-405904"  # <--- REPLACE THIS
LOCATION = "us-central1"

# Versions
PROJECT_TAG = "csv"
SOURCE_VERSION = "001" # <--- CHANGE THIS TO THE CURRENT Transcripts Validator TABLE VERSION
DATA_VERSION = "005" # <--- CHANGE THIS TO THE CURRENT Synthetic Data TABLE VERSION
SCRIPT_VERSION = "003"
DESTINATION_TABLE_VERSION = "003"

# Resources
DATASET_ID = f"vel_{PROJECT_TAG}_schema"
TRANSCRIPTION_TABLE = f"vel_{PROJECT_TAG}_synthetic_transcripts_{DATA_VERSION}"
SOURCE_TABLE = f"vel_{PROJECT_TAG}_transcripts_validation_{SOURCE_VERSION}"
DESTINATION_TABLE = f"vel_{PROJECT_TAG}_derived_signals_{DESTINATION_TABLE_VERSION}"

# Initialize Vertex AI and BigQuery
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)
model = GenerativeModel("gemini-2.5-flash")

### 2. The system prompt

This prompt configures the LLM to act as a "Senior Analyst."

It performs three critical tasks on the raw transcripts:

1. Mining "Organic Signals": Detecting unstructured risks/opportunities (e.g., "Creator is burnout").

2. Signal Classification: Actionability for Youtube or out of their control.

3. Scoring SPM Effectiveness: Grading the manager's performance (0-100) based on resolution and empathy.

Output is forced into strict JSON for direct database insertion.

In [ ]:
# ==========================================
# 2. THE SYSTEM PROMPT
# ==========================================
PROMPT_SIGNAL_EXTRACTOR = """
You are a Senior Data and Behavior Analyst at YouTube, specializing in the "Creator Economy." Your job is to audit call transcripts between a Strategic Partner Manager (SPM) and a Content Creator.

You have three primary objectives:

1. ORGANIC SIGNAL DETECTION:
   - Identify critical themes regarding the Creator's status, risk, or opportunity.
   - Examples: "Churn Risk", "Copyright Confusion", "Interest in Shopping", "Burnout", "Payment Issues".

2. SIGNAL CLASSIFICATION (ACTIONABILITY):
   - For each signal, determine if it is **"Actionable"** or **"Non-Actionable"** for YouTube.
   - **Actionable:** YouTube can do something about it. (e.g., fix a bug, clarify a policy, escalate a support ticket, offer a beta feature, provide training).
   - **Non-Actionable:** Issues out of YouTube's control. (e.g., Creator's personal fatigue unrelated to platform, market competition generic trends, weather, personal life).

3. SPM SCORING:
   - Rate the SPM's effectiveness (1-100) based on Empathy, Clarity, and Resolution.

OUTPUT FORMAT (STRICT JSON):
Respond ONLY with a valid JSON object:
{
  "spm_score": [0-100],
  "spm_reasoning": "Explanation...",
  "signals": [
    {
      "signal_name": "Short Name (e.g., Monetization Bug)",
      "category": "Risk | Opportunity | Product_Feedback | Operational",
      "actionability": "Actionable | Non-Actionable",
      "description": "Brief description of the issue",
      "evidence_quote": "Verbatim quote from the creator"
    }
  ],
  "recommended_next_step": "Suggested action..."
}
"""

### 3. Data ingestion (read from bq)

In [ ]:
# ==========================================
# 3. DATA INGESTION (READ FROM BQ)
# ==========================================
def get_transcripts_from_bq(limit=50):
    """
    Fetches transcripts from BigQuery that have not been processed yet.
    Handles cases where the destination table might not exist during the first run.
    """

    # Reference for the destination table to check for existing records
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{DESTINATION_TABLE}"

    # SQL snippet to exclude already processed IDs
    filter_already_processed = f"""
        AND v.conversation_id NOT IN (
            SELECT DISTINCT transcript_id FROM `{table_id}`
        )
    """

    # Check if the destination table exists to avoid "Table not found" errors in the query
    try:
        client.get_table(table_id)
        print(f"🔍 Destination table detected. Filtering out already processed records...")
    except NotFound:
        # If the table is not found, we assume it's the first run and skip the filter
        print(f"ℹ️ Destination table does not exist yet. Processing all available records.")
        filter_already_processed = ""

    # Main query to fetch raw transcripts and metadata
    query = f"""
        SELECT
            v.conversation_id as transcript_id,
            v.creator_id,
            v.spm_name,
            v.creator_region,
            d.raw_transcript as transcript
        FROM `{PROJECT_ID}.{DATASET_ID}.{SOURCE_TABLE}` v
        INNER JOIN `{PROJECT_ID}.{DATASET_ID}.{TRANSCRIPTION_TABLE}` d
          ON v.conversation_id = d.conversation_id
        WHERE v.is_valid
        {filter_already_processed}
        LIMIT {limit}
    """

    try:
        print(f"📥 Fetching data from BigQuery...")
        df = client.query(query).to_dataframe()
        return df
    except Exception as e:
        print(f"❌ Error reading from BigQuery: {e}")
        return pd.DataFrame()

### 4. Analysis Engine (LLM processing)

In [ ]:
# ==========================================
# 4. ANALYSIS ENGINE (LLM PROCESSING)
# ==========================================
def analyze_transcripts(df):
    results = []
    print(f"🧠 Analyzing {len(df)} rows with Gemini...")

    for index, row in df.iterrows():
        try:
            user_prompt = f"""
            METADATA:
            - TRANSCRIPT_ID: {row['transcript_id']}
            - SPM NAME: {row['spm_name']}
            - CREATOR ID: {row['creator_id']}
            - CREATOR REGION: {row['creator_region']}

            TRANSCRIPT TEXT:
            {row['transcript']}
            """

            response = model.generate_content(
                [PROMPT_SIGNAL_EXTRACTOR, user_prompt],
                generation_config=GenerationConfig(
                    temperature=0.0,
                    response_mime_type="application/json"
                )
            )

            analysis = json.loads(response.text)

            for signal in analysis.get('signals', []):
                results.append({
                    "transcript_id": row['transcript_id'],
                    "spm_name": row['spm_name'],
                    "creator_id": row['creator_id'],
                    "creator_region": row['creator_region'],
                    "spm_score": analysis.get('spm_score'),
                    "spm_reasoning": analysis.get('spm_reasoning'),

                    # --- SIGNAL FIELDS ---
                    "signal_name": signal.get('signal_name'),
                    "signal_category": signal.get('category'),
                    "signal_actionability": signal.get('actionability'),
                    "signal_description": signal.get('description'),
                    "signal_evidence": signal.get('evidence_quote', signal.get('evidence')),

                    "recommended_action": analysis.get('recommended_next_step')
                })

            print(f"  -> Processed {row['transcript_id']} | Signals found: {len(analysis.get('signals', []))}")
            print(user_prompt)
            time.sleep(0.5)

        except Exception as e:
            print(f"❌ Error analyzing {row['transcript_id']}: {e}")
            continue

    return pd.DataFrame(results)

### 5. Upload Results to BigQuery

In [ ]:
# ==========================================
# 5. UPLOAD RESULTS TO BIGQUERY
# ==========================================
def initialize_destination_table():
    """
    Creates the destination table if it doesn't exist yet to prevent JOIN errors.
    """
    table_ref = f"{PROJECT_ID}.{DATASET_ID}.{DESTINATION_TABLE}"

    # Define the strict schema here
    schema = [
        bigquery.SchemaField("transcript_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("spm_name", "STRING"),
        bigquery.SchemaField("creator_id", "STRING"),
        bigquery.SchemaField("creator_region", "STRING"),
        bigquery.SchemaField("spm_score", "INTEGER"),
        bigquery.SchemaField("spm_reasoning", "STRING"),
        bigquery.SchemaField("signal_name", "STRING"),
        bigquery.SchemaField("signal_category", "STRING"),
        bigquery.SchemaField("signal_actionability", "STRING"),
        bigquery.SchemaField("signal_description", "STRING"),
        bigquery.SchemaField("signal_evidence", "STRING"),
        bigquery.SchemaField("recommended_action", "STRING"),
        bigquery.SchemaField("processed_at", "TIMESTAMP")
    ]

    try:
        client.get_table(table_ref)
        print(f"✅ Destination table {DESTINATION_TABLE} exists.")
    except NotFound:
        print(f"⚠️ Destination table not found. Creating {DESTINATION_TABLE}...")
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)
        print("✅ Table created successfully.")


def upload_signals_to_bq(df):
    if df.empty:
        print("⚠️ No signals detected. Nothing to upload.")
        return

    # Add timestamp
    df['processed_at'] = pd.Timestamp.now()

    table_ref = f"{PROJECT_ID}.{DATASET_ID}.{DESTINATION_TABLE}"

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        # Use the exact same schema defined in the init function
        schema=[
            bigquery.SchemaField("transcript_id", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("spm_name", "STRING"),
            bigquery.SchemaField("creator_id", "STRING"),
            bigquery.SchemaField("creator_region", "STRING"),
            bigquery.SchemaField("spm_score", "INTEGER"),
            bigquery.SchemaField("spm_reasoning", "STRING"),
            bigquery.SchemaField("signal_name", "STRING"),
            bigquery.SchemaField("signal_category", "STRING"),
            bigquery.SchemaField("signal_actionability", "STRING"),
            bigquery.SchemaField("signal_description", "STRING"),
            bigquery.SchemaField("signal_evidence", "STRING"),
            bigquery.SchemaField("recommended_action", "STRING"),
            bigquery.SchemaField("processed_at", "TIMESTAMP")
        ]
    )

    print(f"🚀 Uploading {len(df)} detected signals to {table_ref}...")
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()
    print("✅ Upload Complete!")

## Run Main

In [ ]:
if __name__ == "__main__":
    start_time = time.perf_counter()

    # 1. Initialize Destination Table (Crucial Step)
    initialize_destination_table()

    # 2. Read transcriptions
    df_transcripts = get_transcripts_from_bq(limit=120)
    end_time_read = time.perf_counter()

    # 3. Analyze with LLM
    df_signals = analyze_transcripts(df_transcripts)
    end_time_llm = time.perf_counter()

    # 4. Save Results
    upload_signals_to_bq(df_signals)
    end_time = time.perf_counter()

    # Execution times
    duration_read = str(timedelta(seconds=end_time_read - start_time))
    duration_llm = str(timedelta(seconds=end_time_llm - end_time_read))
    duration_save = str(timedelta(seconds=end_time - end_time_llm))
    duration_total = str(timedelta(seconds=end_time - start_time))

    print(f"Read BigQuery execution time (HH:MM:SS): {duration_read}")
    print(f"LLM Analysis execution time (HH:MM:SS): {duration_llm}")
    print(f"Save signals to BigQuery execution time (HH:MM:SS): {duration_save}")
    print(f"Total execution time (HH:MM:SS): {duration_total}")

    # Preview
    print("\n--- PREVIEW OF DETECTED SIGNALS ---")
    try:
      print(df_signals[['spm_name', 'signal_name', 'spm_score']].head().to_markdown(index=False))
    except:
      print(df_signals.head().to_markdown(index=False))

⚠️ Destination table not found. Creating vel_csv_derived_signals_003...
✅ Table created successfully.
🔍 Destination table detected. Filtering out already processed records...
📥 Fetching data from BigQuery...
🧠 Analyzing 73 rows with Gemini...
  -> Processed 1_0_1771463705 | Signals found: 5
  -> Processed 8_1_1771464210 | Signals found: 6
  -> Processed 30_0_1771465427 | Signals found: 6
  -> Processed 3_0_1771463776 | Signals found: 13
  -> Processed 3_1_1771463831 | Signals found: 12
  -> Processed 1_0_1771530949 | Signals found: 7
  -> Processed 72_1_1771467932 | Signals found: 7
  -> Processed 1_1_1771530986 | Signals found: 12
  -> Processed 39_0_1771465931 | Signals found: 12
  -> Processed 28_0_1771465305 | Signals found: 14
  -> Processed 41_0_1771466063 | Signals found: 11
  -> Processed 72_0_1771467872 | Signals found: 7
  -> Processed 8_0_1771464161 | Signals found: 14
  -> Processed 60_1_1771467116 | Signals found: 10
  -> Processed 38_1_1771465902 | Signals found: 10
  -> 

In average it takes ~20 seconds to extract signals from a transcription